In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import numpy as np

In [2]:
MobileNet = keras.applications.mobilenet.MobileNet(input_shape=(160,160,3),weights='imagenet',include_top=False)
# MobileNet.summary()
# Inception = keras.applications.inception_v3.InceptionV3(input_shape=(224,224,3),include_top=False,weights='imagenet')
# Inception.summary()
# MobileNet_v2 = keras.applications.mobilenet_v2.MobileNetV2()
# MobileNet_v2.summary()

In [3]:
# model transfer
for layer in MobileNet.layers:
    layer.trainable = False

last_output = MobileNet.output
Connect = layers.Flatten()(last_output)
Connect = layers.Dense(1024, activation='relu')(Connect)
Connect = layers.Dropout(0.5, name='additionalDrop1')(Connect)
Connect = layers.Dense(1024, activation='relu')(Connect)
Connect = layers.Dropout(0.5, name='additionalDrop2')(Connect)
Connect = layers.Dense(10, activation='softmax')(Connect)
New_mobileNet = keras.models.Model(MobileNet.input,Connect)
    
New_mobileNet.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 161, 161, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 80, 80, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 80, 80, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 80, 80, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 80, 80, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 80, 80, 32)        128   

In [4]:
train_data_path = './Animals_datasets/raw-img'

train_data = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input,
                                validation_split=0.2
                               )

train_generator = train_data.flow_from_directory(train_data_path,
                                                subset = 'training',
                                                target_size=(160,160),
                                                batch_size=16,
                                                class_mode = 'categorical',
                                                color_mode='rgb',
                                                shuffle=True
                                                )

validation_generator = train_data.flow_from_directory(train_data_path,
                                                subset='validation',
                                                target_size=(160,160),
                                                batch_size=16,
                                                class_mode = 'categorical',
                                                color_mode='rgb',
                                                shuffle=True
                                                )

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [5]:
New_mobileNet.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['Accuracy']
                 )

history = New_mobileNet.fit_generator(generator=train_generator,
                                  steps_per_epoch=train_generator.n/train_generator.batch_size,
                                 
                                  validation_data=validation_generator,
                                  validation_steps=validation_generator.n/validation_generator.batch_size,
                                 
                                  epochs=3,
                                  verbose=1
                                 )

Epoch 1/3
1310/1309 [==============================] - 901s 688ms/step - loss: 13.0105 - Accuracy: 0.8294 - val_loss: 13.1237 - val_Accuracy: 0.8372
Epoch 2/3
1310/1309 [==============================] - 906s 692ms/step - loss: 13.1372 - Accuracy: 0.8368 - val_loss: 13.1483 - val_Accuracy: 0.8369
Epoch 3/3
1310/1309 [==============================] - 899s 686ms/step - loss: 13.1488 - Accuracy: 0.8368 - val_loss: 13.1483 - val_Accuracy: 0.8369
